### Step 1: Import Required Libraries

If the following code doesn't run, then do 'pip install ipynb' in the command line. This code lets us import functions from notebooks in the lib folder. Lib has all of the feature extraction and model training/predicting functions.

In [1]:
import ipynb
import sys
sys.path.append('../lib/')

If the following code doesn't run, then do 'pip install imblearn' in the command line. This code lets us do SMOTE (synthetic minority oversampling technique) and random undersampling to help deal with the imbalanced data.

In [2]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

Here we import the remaining libraries that we'll need.

In [3]:
import pandas as pd
import numpy as np
import math
import os
import scipy.io
import pickle
import bz2
import time
import _pickle as cPickle
from sklearn.metrics import pairwise_distances, classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score

### Step 2: Set Work Directories

In [4]:
np.random.seed(2020)

Here we set the directories for the training set points and labels.

In [5]:
train_dir = '../data/train_set/'
train_image_dir = train_dir+"images/"
train_pt_dir = train_dir+"points/"
train_label_path = train_dir+"label.csv"

### Step 3: Set Up Controls

In this cell, we have a set of controls for the feature extraction. If true, then we process the features from scratch, and if false, then we load existing features from files in the output folder. 

+ (T/F) initial feature extraction on training set
+ (T/F) initial feature extraction on test set

+ (T/F) improved feature extraction on training set
+ (T/F) improved feature extraction on test set

+ (T/F) SMOTE using improved features on train set

+ (T/F) PCA using improved features on training set
+ (T/F) PCA using improved features on test set

In [6]:
run_feature_train_initial = True
run_feature_test_initial = True

run_feature_train = True 
run_feature_test = True 

run_feature_train_SMOTE = True

run_feature_train_PCA = True
run_feature_test_PCA = True

In this cell, we have a set of controls for model training/testing. If true, then we train the model and generate predictions on the test set, and if false, then we skip that model. By default only the baseline and advanced models are set to run, but you can set the other models to be True to see how they perform.

In [22]:
run_baseline = True
run_advanced = True

run_baseline_improved = True
run_baseline_pca = True
run_knn = True
run_knn_smote = True
run_xgboost=True
feature_initial=True
run_random_forest=True
run_LDA=True
run_logistic=True
run_svm = True
run_lasso = True
run_bagging_smote = True
run_naivebayes = True

### Step 4: Import Data and Train-Test Split

Here we import the data, and we can see that the dataset is imbalanced and that there are more records with basic emotions than records with complex emotions.

In [8]:
info = pd.read_csv(train_label_path)
n = info.shape[0]

#Data is imbalanced 
print('Number of records with label 0 (basic emotion):   {:4d} '.format(info.loc[info['label']==0].shape[0]))
print('Number of records with label 1 (complex emotion): {:2d} '.format(info.loc[info['label']==1].shape[0]))

Number of records with label 0 (basic emotion):   2402 
Number of records with label 1 (complex emotion): 598 


We do an 80-20 train-test split.

In [9]:
n_train = int(round(n*(4/5),0))
train_idx = np.random.choice(list(info.index),size=n_train,replace=False)
test_idx = list(set(list(info.index))-set(train_idx)) #set difference

Fiducial points are stored in matlab format. In this step, we read them and store them in a list.

In [10]:
#function to read fiducial points
#input: index
#output: matrix of fiducial points corresponding to the index

n_files = len(os.listdir(train_pt_dir))

def readMat_matrix(index):
    try:
        mat_data = scipy.io.loadmat(train_pt_dir+'{:04d}'.format(index)+'.mat')['faceCoordinatesUnwarped']
    except KeyError:
        mat_data = scipy.io.loadmat(train_pt_dir+'{:04d}'.format(index)+'.mat')['faceCoordinates2']
    return np.matrix.round(mat_data,0)

#load fiducial points into list and store them in output
fiducial_pt_list = list(map(readMat_matrix,list(range(1,n_files+1))))
pickle.dump(fiducial_pt_list, open( "../output/fiducial_pt_list.p", "wb" ) )

### Step 5: Construct Features and Responses

#### Starter Code Features

Use feature.ipynb's feature_initial function to generate pairwise distance features for the baseline model. This is the same feature extraction method as that of the starter code. Note that this method double counts distances between points. That is, there are separate entries for the distance from point A to point B and the distance from point B to point A even though the distances are the same.

Feature extraction times exclude the time it takes to write to an output file.

In [11]:
from ipynb.fs.full.feature import feature_initial

tm_feature_train_intitial = np.nan
if run_feature_train_initial == True:
    start = time.time()
    dat_train_initial = feature_initial(fiducial_pt_list, train_idx, info)
    end = time.time()
    tm_feature_train_initial = end-start
    with bz2.BZ2File('../output/train_data_initial' + '.pbz2', 'w') as f: 
        cPickle.dump(dat_train_initial, f)
    print('Initial feature extraction time for train: {:4f}'.format(tm_feature_train_initial))
else:
    dat_train_initial = cPickle.load(bz2.BZ2File('../output/train_data_initial.pbz2', 'rb'))
        
        
tm_feature_test_initial = np.nan
if run_feature_test_initial == True:
    start = time.time()
    dat_test_initial = feature_initial(fiducial_pt_list, test_idx, info)
    end = time.time()
    tm_feature_test_initial = end-start
    with bz2.BZ2File('../output/test_data_initial' + '.pbz2', 'w') as f: 
        cPickle.dump(dat_test_initial, f)
    print('Initial feature extraction time for test:  {:4f}'.format(tm_feature_test_initial))
else:
    dat_test_initial = cPickle.load(bz2.BZ2File('../output/test_data_initial.pbz2', 'rb'))

Initial feature extraction time for train: 5.113635
Initial feature extraction time for test:  1.275087


In [12]:
feature_train_initial = dat_train_initial.loc[:, dat_train_initial.columns != 'labels']
label_train_initial = dat_train_initial['labels']

feature_test_initial = dat_test_initial.loc[:, dat_test_initial.columns != 'labels']
label_test_initial = dat_test_initial['labels'] 

#### Improved Features

Use feature's feature_improved function to generate pairwise distance features to be used by all of the models other than the baseline. Unlike feature_initial, feature_improved does not double count distances. Hence, feature_improved produces exactly half as many features as feature_initial while keeping the same information.

In [13]:
from ipynb.fs.full.feature import feature_improved

tm_feature_train_improved = np.nan
if run_feature_train == True:
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    end = time.time()
    tm_feature_train_improved = end-start
    with bz2.BZ2File('../output/train_data' + '.pbz2', 'w') as f: 
        cPickle.dump(dat_train, f)
    print('Improved feature extraction time for train: {:4f}'.format(tm_feature_train_improved))
else:
    dat_train = cPickle.load(bz2.BZ2File('../output/train_data.pbz2', 'rb'))


tm_feature_test_improved = np.nan
if run_feature_test == True:
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    end = time.time()
    tm_feature_test_improved = end-start
    with bz2.BZ2File('../output/test_data' + '.pbz2', 'w') as f: 
        cPickle.dump(dat_test, f)
    print('Improved feature extraction time for test:  {:4f}'.format(tm_feature_test_improved))
else:
    dat_test = cPickle.load(bz2.BZ2File('../output/test_data.pbz2', 'rb'))

Improved feature extraction time for train: 0.183609
Improved feature extraction time for test:  0.039230


In [14]:
feature_train = dat_train.loc[:, dat_train.columns != 'labels']
label_train = dat_train['labels'] 

feature_test = dat_test.loc[:, dat_test.columns != 'labels']
label_test = dat_test['labels']

#### SMOTE Features

Here we do the feature extraction for SMOTE which will be discussed more in the advanced model section. SMOTE is only done on the training data and not on the test data. SMOTE is a modification of the improved features.

In [17]:
from ipynb.fs.full.feature import feature_SMOTE

tm_feature_train_SMOTE = np.nan
if run_feature_train_SMOTE == True:
    start = time.time()
    dat_train_SMOTE = feature_SMOTE(fiducial_pt_list, train_idx, info)
    end = time.time()
    tm_feature_train_SMOTE = end-start
    with bz2.BZ2File('../output/train_data_SMOTE' + '.pbz2', 'w') as f: 
        cPickle.dump(dat_train, f)
    print('SMOTE feature extraction time for train: {:4f}'.format(tm_feature_train_SMOTE))
else:
    dat_train_SMOTE = cPickle.load(bz2.BZ2File('../output/train_data_SMOTE.pbz2', 'rb'))

SMOTE feature extraction time for train: 2.474462


In [18]:
feature_train_sm = dat_train_SMOTE.loc[:,dat_train_SMOTE.columns!='labels']
label_train_sm = dat_train_SMOTE['labels']

#### PCA Features

Finally, here we do PCA which is only done for the model candidates that were not chosen for the advanced model. PCA is done as a modification of the improved features.

In [17]:
from ipynb.fs.full.feature import feature_PCA

tm_feature_train_PCA = np.nan
if run_feature_train_PCA == True:
    start = time.time()
    feature_train_scaled = dat_train.loc[:, dat_train.columns != 'labels']
    scaler = MinMaxScaler()
    feature_train_scaled = scaler.fit_transform(feature_train)

#pick the number of components that captures 95% of the variance
pca = PCA(n_components = 0.95, svd_solver='full').fit(feature_train_scaled)
feature_train_PCA = pca.transform(feature_train_scaled)

tm_feature_test_PCA = np.nan
if run_feature_test_PCA == True:
    feature_test_scaled = scaler.fit_transform(feature_test)
    feature_test_PCA = pca.transform(feature_test_scaled)


### Step 6: Baseline Model

Before discussing the models, we will go over the two main metrics we used for finding a better model than the baseline. Since the data is imbalanced, we examined AUC and balanced accuracy rather than the regular accuracy metric.

AUC is the area under the ROC curve which measures the TP vs FP rate as the classification decision threshold changes over time.

Balanced accuracy is given by the formula $$balanced\_accuracy = \frac{1}{2}(\frac{TP}{TP+FN}+\frac{TN}{TN+FP}).$$ 


Balanced accuracy is used for imbalanced data as an estimate for the accuracy if the data was balanced, so the true performance of our models on balanced data will be close to the balanced accuracy.

The baseline model is a GBM fitted on the initial pairwise fiducial features. The parameters were chosen from a grid search with AUC scoring.

In [18]:
#data frame used to store all of the model results
model_results_df = pd.DataFrame(columns=['Feature Extraction Train Time','Feature Extraction Test Time',
                                         'Train Time','Prediction Time','Accuracy','Balanced Accuracy','AUC'])

In [19]:
#grid search for optimal parameters
#params = {'learning_rate':[0.01,0.05,0.1,0.5], 'max_depth': [1,2,3], 'n_estimators':[50,100,150]}
#gscv = GridSearchCV(GradientBoostingClassifier(),params,cv=3,scoring='roc_auc').fit(feature_train_initial,label_train_initial)
#gscv.best_params_
# output: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150}

In [26]:
from ipynb.fs.full.train_gbm import train_gbm
from ipynb.fs.full.test_model import test_model
from ipynb.fs.full.compute_metrics import compute_metrics

if run_baseline == True:
    
    [train_time,baseline] = train_gbm(feature_train_initial,label_train_initial,
                                      learning_rate=0.1,max_depth=3,n_estimators=150)
    print('Training time: {:4f} seconds'.format(train_time))
    
    [prediction_time,test_preds] = test_model(baseline,feature_test_initial)
    print('Prediction time: {:4f} seconds'.format(prediction_time))
    
    [accuracy, balanced_accuracy, auc] = compute_metrics(feature_test_initial,label_test_initial,test_preds,baseline)
    print('\nAccuracy: {:4f}'.format(accuracy))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy))
    print('AUC: {:4f}'.format(auc))
    
    #save baseline model
    pickle.dump(baseline,open("../output/baseline.p", "wb"))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train_initial,
                    'Feature Extraction Test Time':tm_feature_test_initial,
                     'Train Time':train_time,
                     'Prediction Time':prediction_time,
                    'Accuracy':accuracy,
                    'AUC':auc,
                    'Balanced Accuracy':balanced_accuracy},name='Baseline')
    model_results_df = model_results_df.append(row)

Training time: 189.633951 seconds
Prediction time: 0.054330 seconds

Accuracy: 0.808333
Balanced Accuracy: 0.587563
AUC: 0.785287


### Step 7: Advanced Model (SMOTEBoost)

For the advanced model, we decided to use SMOTEBoost, which is a modified version of XGBoost that uses SMOTE (Synethic Minority Oversampling Technique) (add reference). Our model also uses the improved features which do not double count distances, and the parameters were chosen from grid search with AUC scoring.

The idea of SMOTE is to modify the imbalanced training data by first randomly undersampling from the majority class and then creating new synthetic minority data that is close to the existing feature space. The modified SMOTE features then have equal number of data in each class.

We went with this model for a couple of reasons. First of all, it addresses the fact that the training data is imbalanced. It also has a higher AUC, accuracy, and balanced accuracy than the baseline GBM model. Finally, compared to the other candidates for the advanced model, it has the highest AUC and balanced accuracy from 10 fold cross validation.

In [19]:
from ipynb.fs.full.train_xgb import train_xgb
from ipynb.fs.full.test_model import test_model
from ipynb.fs.full.compute_metrics import compute_metrics

if run_advanced == True:
    
    [train_time, advanced] = train_xgb(feature_train_sm, label_train_sm, learning_rate=0.25, n_estimators=300,
                                      max_depth=3,min_child_weight=1,objective='binary:logistic',scale_pos_weight=4)
    print('Training time: {:4f} seconds'.format(train_time))

         
    [prediction_time,test_preds] = test_model(advanced,feature_test)
    print('Prediction time: {:4f} seconds'.format(prediction_time))
    
    [accuracy, balanced_accuracy, auc] = compute_metrics(feature_test,label_test,test_preds,advanced)
    print('\nAccuracy: {:4f}'.format(accuracy))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy))
    print('AUC: {:4f}'.format(auc))
    
    pickle.dump(advanced,open("../output/advanced.p", "wb"))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train_SMOTE,
                    'Feature Extraction Test Time':tm_feature_test_improved,
                     'Train Time':train_time,
                     'Prediction Time':prediction_time,
                    'Accuracy':accuracy,
                    'AUC':auc,
                    'Balanced Accuracy':balanced_accuracy},name='Advanced (SMOTEBoost)')
    model_results_df.loc['Advanced (SMOTEBoost)'] = row    

Training time: 211.646848 seconds
Prediction time: 0.077861 seconds

Accuracy: 0.821667
Balanced Accuracy: 0.670898
AUC: 0.821778


NameError: name 'model_results_df' is not defined

### Optional Step: Remaining Models

These are the other models that were candidates for the advanced model. (By default they are set to not run to save time on running this notebook).

#### Baseline Model with Improved Features

In [ ]:
#grid search for optimal parameters
#params = {'learning_rate':[0.01,0.05,0.1,0.5], 'max_depth': [1,2,3], 'n_estimators':[50,100,150]}
#gscv = GridSearchCV(GradientBoostingClassifier(),params,cv=3,scoring='roc_auc').fit(feature_train,label_train)
#gscv.best_params_
# output: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150}

In [27]:
if run_baseline_improved == True:
    
    [train_time, baseline_improved] = train_gbm(feature_train,label_train,
                                                learning_rate=0.1,max_depth=3,n_estimators=150)
    
    print('Training time: {:4f} seconds'.format(train_time))
    
    [prediction_time,test_preds] = test_model(baseline_improved,feature_test)
    print('Prediction time: {:4f} seconds'.format(prediction_time))
    
    [accuracy, balanced_accuracy, auc] = compute_metrics(feature_test_initial,label_test_initial,test_preds,baseline)
    print('\nAccuracy: {:4f}'.format(accuracy))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy))
    print('AUC: {:4f}'.format(auc))

    row = pd.Series({'Feature Extraction Train Time':tm_feature_train_improved,
                'Feature Extraction Test Time':tm_feature_test_improved,
                 'Train Time':train_time,
                 'Prediction Time':prediction_time,
                'Accuracy':accuracy,
                'AUC':auc,
                'Balanced Accuracy':balanced_accuracy},name='Baseline with Improved Features')
    model_results_df = model_results_df.append(row)

Training time: 190.495405 seconds
Prediction time: 0.027578 seconds

Accuracy: 0.816667
Balanced Accuracy: 0.610128
AUC: 0.785287


### Baseline Model With PCA

In [20]:
if run_baseline_pca == True:
    
    tm_feature_train = np.nan
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    feature_train = dat_train.loc[:, dat_train.columns != 'labels']
    label_train = dat_train['labels']
    scaler = MinMaxScaler()
    feature_train_scaled = scaler.fit_transform(feature_train)
    #pick the number of components that captures 95% of the variance
    pca = PCA(n_components = 0.95, svd_solver='full').fit(feature_train_scaled)
    feature_train_PCA = pca.transform(feature_train_scaled)
    end = time.time()
    tm_feature_train = end-start
         
    tm_feature_test = np.nan
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    feature_test = dat_test.loc[:, dat_test.columns != 'labels']
    label_test = dat_test['labels']
    feature_test_PCA = pca.transform(feature_test_scaled)
    end = time.time()
    tm_feature_test = end-start
    
    #params = {'learning_rate':[0.01,0.05,0.1,0.5], 'max_depth': [1,2,3], 'n_estimators':[50,100,150]}
    #gscv = GridSearchCV(GradientBoostingClassifier(),params,cv=3).fit(feature_train_PCA,label_train)
    #gscv.best_params_
    #output: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 150}

    start = time.time()
    gbm_pca=GradientBoostingClassifier(learning_rate=0.1,max_depth=2,n_estimators=150)
    gbm_pca.fit(feature_train_PCA,label_train)
    end = time.time()
    train_time = end-start
    print('Training time: {:4f} seconds'.format(end-start))
    
    start = time.time()
    test_preds = gbm_pca.predict(feature_test_PCA)
    end = time.time()
    prediction_time = end-start
    print('Prediction time: {:4f} seconds'.format(end-start))
    
    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    accuracy = 1-classification_error
    balanced_accuracy = balanced_accuracy_score(label_test,test_preds)
    test_probs = gbm_pca.predict_proba(feature_test_PCA)[:,1]
    auc = roc_auc_score(label_test,test_probs)
    
    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy_score(label_test,test_preds)))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_probs)))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train,
            'Feature Extraction Test Time':tm_feature_test,
             'Train Time':train_time,
             'Prediction Time':prediction_time,
            'Accuracy':accuracy,
            'AUC':auc,
            'Balanced Accuracy':balanced_accuracy},name='Baseline with PCA')
    model_results_df = model_results_df.append(row)

Training time: 1.319941 seconds
Prediction time: 0.001212 seconds

Classification Error: 0.213333
Accuracy: 0.786667

Balanced Accuracy: 0.527742
Classification Report:

              precision    recall  f1-score   support

           0       0.80      0.98      0.88       473
           1       0.48      0.08      0.14       127

    accuracy                           0.79       600
   macro avg       0.64      0.53      0.51       600
weighted avg       0.73      0.79      0.72       600

Confusion Matrix:

[[462  11]
 [117  10]]

AUC: 0.691332


### KNN Model

In [ ]:
#params = {'n_neighbors':list(range(5,55,5))}
#gscv = GridSearchCV(KNeighborsClassifier(),params,cv=5).fit(feature_train,label_train)
#gscv.best_params_
#output: {'n_neighbors': 25}

In [30]:
from ipynb.fs.full.train_knn import train_knn

if run_knn == True:
    
    [train_time, knn] = train_knn(feature_train,label_train,n_neighbors=25)
    print('Training time: {:4f} seconds'.format(train_time))
    
    [prediction_time,test_preds] = test_model(knn,feature_test)
    print('Prediction time: {:4f} seconds'.format(prediction_time))
    
    [accuracy, balanced_accuracy, auc] = compute_metrics(feature_test,label_test,test_preds,knn)
    print('\nAccuracy: {:4f}'.format(accuracy))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy))
    print('AUC: {:4f}'.format(auc))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train_improved,
        'Feature Extraction Test Time':tm_feature_test_improved,
         'Train Time':train_time,
         'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='KNN')
    model_results_df = model_results_df.append(row)

Training time: 0.397339 seconds
Prediction time: 5.921443 seconds

Accuracy: 0.791667
Balanced Accuracy: 0.516514
AUC: 0.674527


### SMOTE KNN

In [ ]:
#params = {'n_neighbors':list(range(5,55,5))}
#gscv = GridSearchCV(KNeighborsClassifier(),params,cv=5).fit(feature_train_sm,label_train_sm)
#gscv.best_params_
#output: {'n_neighbors': 5}

In [31]:
if run_knn_smote == True:
    
    [train_time, knn] = train_knn(feature_train_sm,label_train_sm,n_neighbors=5)
    print('Training time: {:4f} seconds'.format(train_time))
    
    [prediction_time,test_preds] = test_model(knn,feature_test)
    print('Prediction time: {:4f} seconds'.format(prediction_time))
    
    [accuracy, balanced_accuracy, auc] = compute_metrics(feature_test,label_test,test_preds,knn)
    print('\nAccuracy: {:4f}'.format(accuracy))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy))
    print('AUC: {:4f}'.format(auc))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train_SMOTE,
        'Feature Extraction Test Time':tm_feature_test_improved,
         'Train Time':train_time,
         'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='SMOTE KNN')
    model_results_df = model_results_df.append(row)

Training time: 0.771669 seconds
Prediction time: 8.114751 seconds

Accuracy: 0.606667
Balanced Accuracy: 0.638211
AUC: 0.683691


### XGBoost Model

In [23]:
if run_xgboost == True:
    
    [train_time, xgb] = train_xgb(feature_train, label_train, learning_rate=0.1, n_estimators=200,
                                      max_depth=3,min_child_weight=1,objective='binary:logistic',scale_pos_weight=4)
    print('Training time: {:4f} seconds'.format(train_time))

         
    [prediction_time,test_preds] = test_model(xgb,feature_test)
    print('Prediction time: {:4f} seconds'.format(prediction_time))

    [accuracy, balanced_accuracy, auc] = compute_metrics(feature_test,label_test,test_preds,xgb)
    print('\nAccuracy: {:4f}'.format(accuracy))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy))
    print('AUC: {:4f}'.format(auc))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train_improved,
        'Feature Extraction Test Time':tm_feature_test_improved,
         'Train Time':train_time,
         'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='XGBoost')
    model_results_df = model_results_df.append(row)

Training time: 89.071407 seconds
Prediction time: 0.079029 seconds

Classification Error: 0.186667
Accuracy: 0.813333

Balanced Accuracy: 0.688652
Classification Report:

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       473
           1       0.57      0.47      0.52       127

    accuracy                           0.81       600
   macro avg       0.72      0.69      0.70       600
weighted avg       0.80      0.81      0.81       600

Confusion Matrix:

[[428  45]
 [ 67  60]]

AUC: 0.808726


### Random Forest Model

In [23]:
from ipynb.fs.full.train_random_forest import train_random_forest

if run_random_forest==True:
    
    [train_time, rf_model] = train_random_forest(feature_train,label_train,n_estimators=100,criterion='gini',min_samples_leaf=1,max_features='sqrt')
    print('Training time: {:4f} seconds'.format(train_time))
    
    [prediction_time,test_preds] = test_model(rf_model,feature_test)
    print('Prediction time: {:4f} seconds'.format(prediction_time))
    
    [accuracy, balanced_accuracy, auc] = compute_metrics(feature_test,label_test,test_preds,rf_model)
    print('\nAccuracy: {:4f}'.format(accuracy))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy))
    print('AUC: {:4f}'.format(auc))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train_improved,
        'Feature Extraction Test Time':tm_feature_test_improved,
         'Train Time':train_time,
         'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='Random Forest')
    model_results_df = model_results_df.append(row)

Training time: 7.173245 seconds
Prediction time: 0.031935 seconds

Accuracy: 0.808333
Balanced Accuracy: 0.567404
AUC: 0.746608


NameError: name 'model_results_df' is not defined

### LDA Model

In [26]:
from ipynb.fs.full.train_lda import train_lda

if run_LDA==True: 
    
    [train_time, lda_model] = train_lda(feature_train, label_train,solver='eigen', shrinkage=.1, n_components=1)
    print('Training time: {:4f} seconds'.format(train_time))
    
    [prediction_time,test_preds] = test_model(lda_model,feature_test)
    print('Prediction time: {:4f} seconds'.format(prediction_time))
    
    [accuracy, balanced_accuracy, auc] = compute_metrics(feature_test,label_test,test_preds,lda_model)
    print('\nAccuracy: {:4f}'.format(accuracy))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy))
    print('AUC: {:4f}'.format(auc))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train_improved,
        'Feature Extraction Test Time':tm_feature_test_improved,
         'Train Time':train_time,
         'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='LDA')
    model_results_df = model_results_df.append(row)

Training time: 6.055222 seconds
Prediction time: 0.019049 seconds

Accuracy: 0.821667
Balanced Accuracy: 0.636339
AUC: 0.786203


NameError: name 'model_results_df' is not defined

### Logistic Model

In [26]:
#grid={"C":[0.001,0.01,0.1,0.25,0.5,1,10]}
#cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
#gscv = GridSearchCV(LogisticRegression(dual=False, fit_intercept=True,
#                   intercept_scaling=1, max_iter=1200000,
#                   multi_class='multinomial', penalty='l2',
#                   solver='lbfgs', tol=0.0001),grid,cv=cv,return_train_score=True)
#gscv.fit(feature_train,label_train)
#gscv.best_params_
#output: {'C': 0.01}

In [27]:
if run_logistic==True:
    
    tm_feature_train = np.nan
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    feature_train = dat_train.loc[:, dat_train.columns != 'labels']
    label_train = dat_train['labels']
    end = time.time()
    tm_feature_train = end-start
         
    tm_feature_test = np.nan
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    feature_test = dat_test.loc[:, dat_test.columns != 'labels']
    label_test = dat_test['labels']
    end = time.time()
    tm_feature_test = end-start
    
    start_time = time.time() 
    lr = LogisticRegression(C=0.01, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=1200000,
                   multi_class='multinomial', penalty='l2',
                   solver='lbfgs', tol=0.0001)
    lr_model = lr.fit(feature_train,label_train)
    end = time.time()
    train_time = end-start_time
    print('Training time: {:4f} seconds'.format(time.time()-start_time))
    
    start_time = time.time()
    test_preds = lr_model.predict(feature_test)
    print('Prediction time: {:4f} seconds'.format(time.time()-start_time))
    end = time.time()
    prediction_time = end-start_time
    
    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    accuracy = 1-classification_error
    balanced_accuracy = balanced_accuracy_score(label_test,test_preds)
    test_probs = lr_model.predict_proba(feature_test)[:,1]
    auc = roc_auc_score(label_test,test_probs)
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy_score(label_test,test_preds)))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_probs)))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train,
        'Feature Extraction Test Time':tm_feature_test,
         'Train Time':train_time,
         'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='Logistic Regression')
    model_results_df = model_results_df.append(row)

Training time: 100.128018 seconds
Prediction time: 0.024578 seconds

Classification Error: 0.178333
Accuracy: 0.821667

Balanced Accuracy: 0.699697
Classification Report:

              precision    recall  f1-score   support

           0       0.87      0.91      0.89       473
           1       0.60      0.49      0.54       127

    accuracy                           0.82       600
   macro avg       0.73      0.70      0.71       600
weighted avg       0.81      0.82      0.81       600

Confusion Matrix:

[[431  42]
 [ 65  62]]

AUC: 0.822310


### Weighted Logistic Model

In [28]:
#grid={"C":[0.001,0.01,0.1,0.25,0.5,1,10]}
#weights = {0:80.0, 1:20.0}
#cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
#gscv = GridSearchCV(LogisticRegression(dual=False, fit_intercept=True,
#                   intercept_scaling=1, max_iter=1200000,
#                   multi_class='multinomial', penalty='l2',
#                   solver='lbfgs', tol=0.0001,class_weight=weights),grid,cv=cv,return_train_score=True)
#gscv.fit(feature_train,label_train)
#gscv.best_params_
#output: {'C': 0.001}

In [29]:
if run_logistic==True:   
    
    tm_feature_train = np.nan
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    feature_train = dat_train.loc[:, dat_train.columns != 'labels']
    label_train = dat_train['labels']
    end = time.time()
    tm_feature_train = end-start
         
    tm_feature_test = np.nan
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    feature_test = dat_test.loc[:, dat_test.columns != 'labels']
    label_test = dat_test['labels']
    end = time.time()
    tm_feature_test = end-start
    
    start_time = time.time() 
    weights = {0:80.0, 1:20.0}
    lr = LogisticRegression(C=0.001, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=120000000000,
                   multi_class='multinomial', penalty='l2',
                   solver='lbfgs', tol=0.0001,class_weight=weights)
    lr_model = lr.fit(feature_train,label_train)
    train_time = time.time()-start_time
    print('Training time: {:4f} seconds'.format(time.time()-start_time))
    
    
    start_time = time.time()
    test_preds = lr_model.predict(feature_test)
    prediction_time = time.time()-start_time
    print('Prediction time: {:4f} seconds'.format(time.time()-start_time))
 
    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    accuracy = 1-classification_error
    balanced_accuracy = balanced_accuracy_score(label_test,test_preds)
    test_probs = lr_model.predict_proba(feature_test)[:,1]
    auc = roc_auc_score(label_test,test_probs)
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy_score(label_test,test_preds)))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_probs)))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train,
        'Feature Extraction Test Time':tm_feature_test,
         'Train Time':train_time,
         'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='Weighted Logistic')
    model_results_df = model_results_df.append(row)

Training time: 128.495774 seconds
Prediction time: 0.023449 seconds

Classification Error: 0.170000
Accuracy: 0.830000

Balanced Accuracy: 0.679063
Classification Report:

              precision    recall  f1-score   support

           0       0.86      0.94      0.90       473
           1       0.65      0.42      0.51       127

    accuracy                           0.83       600
   macro avg       0.76      0.68      0.70       600
weighted avg       0.81      0.83      0.82       600

Confusion Matrix:

[[445  28]
 [ 74  53]]

AUC: 0.822843


/Users/rohan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### SVM

In [30]:
# #grid search with cv 3 to find the best performed parameters
# param= {'C': [0.00001,0.0001,0.001,0.01,1,10],
#        'kernel':['linear', 'rbf', 'poly'],
#        'degree':[2,3,4]}

# gscv = GridSearchCV(SVC(random_state = 2020), param, cv=3, return_train_score=True)
# gscv.fit(feature_train,label_train)
# gscv.best_params_
# #output: {'C': 10, 'degree': 4, 'kernel': 'poly'}

In [31]:
#improved svm using parameters from grid search
if run_svm==True:
    
    tm_feature_train = np.nan
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    feature_train = dat_train.loc[:, dat_train.columns != 'labels']
    label_train = dat_train['labels']
    end = time.time()
    tm_feature_train = end-start
         
    tm_feature_test = np.nan
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    feature_test = dat_test.loc[:, dat_test.columns != 'labels']
    label_test = dat_test['labels']
    end = time.time()
    tm_feature_test = end-start
    
    start_time = time.time() 
    svm = SVC(C=10, kernel='poly', degree=4, random_state=2020,probability=True)
    svm_model = svm.fit(feature_train,label_train)
    train_time = time.time()-start_time
    print('Training time: {:4f} seconds'.format(time.time()-start_time))
    
    start_time = time.time()
    test_preds = svm_model.predict(feature_test)
    print('Prediction time: {:4f} seconds'.format(time.time()-start_time))
    prediction_time = time.time()-start_time

    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    accuracy = 1-classification_error
    balanced_accuracy = balanced_accuracy_score(label_test,test_preds)
    test_probs = svm_model.predict_proba(feature_test)[:,1]
    auc = roc_auc_score(label_test,test_probs)
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy_score(label_test,test_preds)))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_probs)))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train,
        'Feature Extraction Test Time':tm_feature_test,
         'Train Time':train_time,
         'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='SVM')
    model_results_df = model_results_df.append(row)

Training time: 77.353651 seconds
Prediction time: 1.640872 seconds

Classification Error: 0.145000
Accuracy: 0.855000

Balanced Accuracy: 0.683400
Classification Report:

              precision    recall  f1-score   support

           0       0.86      0.98      0.91       473
           1       0.84      0.39      0.53       127

    accuracy                           0.85       600
   macro avg       0.85      0.68      0.72       600
weighted avg       0.85      0.85      0.83       600

Confusion Matrix:

[[464   9]
 [ 78  49]]

AUC: 0.828137


### SVM With PCA

In [32]:
# #grid search with cv 3 to find the best performed parameters
# param= {'C': [0.001,0.01,1,10,15,20],
#        'kernel':['linear', 'rbf', 'poly'],
#        'degree':[2,3,4]}

# gscv = GridSearchCV(SVC(random_state = 2020), param, cv=3, return_train_score=True)
# gscv.fit(feature_train_PCA,label_train)
# gscv.best_params_
# #output: {'C': 10, 'degree': 2, 'kernel': 'rbf'}

In [33]:
#improved svm with PCA
if run_svm==True:    
    
    tm_feature_train = np.nan
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    feature_train = dat_train.loc[:, dat_train.columns != 'labels']
    label_train = dat_train['labels']
    scaler = MinMaxScaler()
    feature_train_scaled = scaler.fit_transform(feature_train)
    #pick the number of components that captures 95% of the variance
    pca = PCA(n_components = 0.95, svd_solver='full').fit(feature_train_scaled)
    feature_train_PCA = pca.transform(feature_train_scaled)
    end = time.time()
    tm_feature_train = end-start
         
    tm_feature_test = np.nan
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    feature_test = dat_test.loc[:, dat_test.columns != 'labels']
    label_test = dat_test['labels']
    feature_test_PCA = pca.transform(feature_test_scaled)
    end = time.time()
    tm_feature_test = end-start
    
    start_time = time.time() 
    svm = SVC(C=10, kernel='rbf', degree=2, random_state=2020,probability=True)
    svm_model = svm.fit(feature_train_PCA,label_train)
    print('Training time: {:4f} seconds'.format(time.time()-start_time))
    
    start_time = time.time()
    test_preds = svm_model.predict(feature_test_PCA)
    print('Prediction time: {:4f} seconds'.format(time.time()-start_time))
    
    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    accuracy = 1-classification_error
    balanced_accuracy = balanced_accuracy_score(label_test,test_preds)
    test_probs = svm_model.predict_proba(feature_test_PCA)[:,1]
    auc = roc_auc_score(label_test,test_probs)
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy_score(label_test,test_preds)))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_probs)))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train,
        'Feature Extraction Test Time':tm_feature_test,
        'Train Time':train_time,
        'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='SVM with PCA')
    model_results_df = model_results_df.append(row)

Training time: 0.726500 seconds
Prediction time: 0.017300 seconds

Classification Error: 0.213333
Accuracy: 0.786667

Balanced Accuracy: 0.585341
Classification Report:

              precision    recall  f1-score   support

           0       0.82      0.93      0.87       473
           1       0.49      0.24      0.32       127

    accuracy                           0.79       600
   macro avg       0.66      0.59      0.60       600
weighted avg       0.75      0.79      0.76       600

Confusion Matrix:

[[442  31]
 [ 97  30]]

AUC: 0.745093


### Weighted SVM

In [34]:
# weights = {0:80.0, 1:20.0}
# params= {'C': [1,10,15,20],
#        'kernel':['linear', 'rbf', 'poly'],
#        'degree':[2,3,4]}
# cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
# gscv = GridSearchCV(SVC(class_weight=weights,random_state = 2020,probability=True), params, cv=3, scoring='roc_auc',verbose=True)
# gscv.fit(feature_train,label_train)
# gscv.best_params_
# #output: output: {'C': 1, 'degree': 4, 'kernel': 'poly'}

In [35]:
#improved svm using parameters from grid search
if run_svm==True:    
    
    tm_feature_train = np.nan
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    feature_train = dat_train.loc[:, dat_train.columns != 'labels']
    label_train = dat_train['labels']
    end = time.time()
    tm_feature_train = end-start
         
    tm_feature_test = np.nan
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    feature_test = dat_test.loc[:, dat_test.columns != 'labels']
    label_test = dat_test['labels']
    end = time.time()
    tm_feature_test = end-start
    
    start_time = time.time() 
    weights = {0:80.0, 1:20.0}
    svm = SVC(C=10, kernel='poly', degree=4, random_state=2020,class_weight=weights,probability=True)
    svm_model = svm.fit(feature_train,label_train)
    print('Training time: {:4f} seconds'.format(time.time()-start_time))
    
    start_time = time.time()
    test_preds = svm_model.predict(feature_test)
    print('Prediction time: {:4f} seconds'.format(time.time()-start_time))
    
    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    accuracy = 1-classification_error
    balanced_accuracy = balanced_accuracy_score(label_test,test_preds)
    test_probs = svm_model.predict_proba(feature_test)[:,1]
    auc = roc_auc_score(label_test,test_probs)
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy_score(label_test,test_preds)))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_probs)))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train,
        'Feature Extraction Test Time':tm_feature_test,
        'Train Time':train_time,
        'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='Weighted SVM')
    model_results_df = model_results_df.append(row)

Training time: 396.494081 seconds
Prediction time: 1.578482 seconds

Classification Error: 0.163333
Accuracy: 0.836667

Balanced Accuracy: 0.717851
Classification Report:

              precision    recall  f1-score   support

           0       0.88      0.92      0.90       473
           1       0.64      0.51      0.57       127

    accuracy                           0.84       600
   macro avg       0.76      0.72      0.73       600
weighted avg       0.83      0.84      0.83       600

Confusion Matrix:

[[437  36]
 [ 62  65]]

AUC: 0.837209


### Naive Bayes

In [36]:
if run_naivebayes == True:
    
    tm_feature_train = np.nan
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    feature_train = dat_train.loc[:, dat_train.columns != 'labels']
    label_train = dat_train['labels']
    end = time.time()
    tm_feature_train = end-start
         
    tm_feature_test = np.nan
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    feature_test = dat_test.loc[:, dat_test.columns != 'labels']
    label_test = dat_test['labels']
    end = time.time()
    tm_feature_test = end-start
    
    start = time.time()
    gnb = GaussianNB()
    gaussian = gnb.fit(feature_train, label_train)
    end = time.time()
    train_time = end-start
    print('Training time: {:4f} seconds'.format(end-start))
    
    start = time.time()
    test_pred = gaussian.predict(feature_test)
    end = time.time()
    prediction_time = end-start
    print('Prediction time: {:4f} seconds'.format(end-start))
    
    classification_error = np.mean(np.array(test_pred) != np.array(label_test))
    accuracy = 1-classification_error
    balanced_accuracy = balanced_accuracy_score(label_test,test_pred)
    test_probs = gaussian.predict_proba(feature_test)[:,1]
    auc = roc_auc_score(label_test,test_probs)
    
    print('\nClassification Error: {:4f}'.format(classification_error))
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy_score(label_test,test_pred)))
    print('Classification Report:\n')
    print(classification_report(label_test,test_pred))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_pred))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_probs)))
    
       
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train,
        'Feature Extraction Test Time':tm_feature_test,
        'Train Time':train_time,
        'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='Naive Bayes')
    model_results_df = model_results_df.append(row)

Training time: 0.128203 seconds
Prediction time: 0.034613 seconds

Classification Error: 0.336667
Accuracy: 0.663333

Balanced Accuracy: 0.628073
Classification Report:

              precision    recall  f1-score   support

           0       0.86      0.69      0.76       473
           1       0.33      0.57      0.42       127

    accuracy                           0.66       600
   macro avg       0.59      0.63      0.59       600
weighted avg       0.74      0.66      0.69       600

Confusion Matrix:

[[326 147]
 [ 55  72]]

AUC: 0.660369


### Lasso

In [37]:
if run_lasso==True:   
    
    tm_feature_train = np.nan
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    feature_train = dat_train.loc[:, dat_train.columns != 'labels']
    label_train = dat_train['labels']
    end = time.time()
    tm_feature_train = end-start
         
    tm_feature_test = np.nan
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    feature_test = dat_test.loc[:, dat_test.columns != 'labels']
    label_test = dat_test['labels']
    end = time.time()
    tm_feature_test = end-start
    
    start_time = time.time() 
    lasso = LogisticRegression(penalty='l1', solver='liblinear')
    lasso_model = lasso.fit(feature_train,label_train)
    train_time = time.time()-start_time
    print('Training time: {:4f} seconds'.format(time.time()-start_time))
    
    start_time = time.time()
    test_preds = lasso_model.predict(feature_test)
    print('Prediction time: {:4f} seconds'.format(time.time()-start_time))
    prediction_time = time.time()-start_time
    
    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    accuracy = 1-classification_error
    balanced_accuracy = balanced_accuracy_score(label_test,test_preds)
    test_probs = lasso_model.predict_proba(feature_test)[:,1]
    auc = roc_auc_score(label_test,test_probs)
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy_score(label_test,test_preds)))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_probs)))

    row = pd.Series({'Feature Extraction Train Time':tm_feature_train,
        'Feature Extraction Test Time':tm_feature_test,
        'Train Time':train_time,
        'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='Lasso')
    model_results_df = model_results_df.append(row)

Training time: 108.838574 seconds
Prediction time: 0.018132 seconds

Classification Error: 0.171667
Accuracy: 0.828333

Balanced Accuracy: 0.692406
Classification Report:

              precision    recall  f1-score   support

           0       0.86      0.93      0.90       473
           1       0.63      0.46      0.53       127

    accuracy                           0.83       600
   macro avg       0.75      0.69      0.71       600
weighted avg       0.81      0.83      0.82       600

Confusion Matrix:

[[439  34]
 [ 69  58]]

AUC: 0.824241


### Weighted Lasso

In [38]:
# weights = {0:80.0, 1:20.0}
# param= {'solver':['liblinear', 'saga']}
# cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
# gscv = GridSearchCV(LogisticRegression(penalty='l1', class_weight=weights), params, cv=3, scoring='roc_auc',verbose=True)
# gscv.fit(feature_train,label_train)
# gscv.best_params_

In [39]:
if run_lasso==True: 
    
    tm_feature_train = np.nan
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    feature_train = dat_train.loc[:, dat_train.columns != 'labels']
    label_train = dat_train['labels']
    end = time.time()
    tm_feature_train = end-start
         
    tm_feature_test = np.nan
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    feature_test = dat_test.loc[:, dat_test.columns != 'labels']
    label_test = dat_test['labels']
    end = time.time()
    tm_feature_test = end-start
    
    weights = {0:80.0, 1:20.0}
    start_time = time.time() 
    lasso_w = LogisticRegression(penalty='l1', solver='liblinear', class_weight=weights)
    lasso_model_w = lasso_w.fit(feature_train,label_train)
    print('Training time: {:4f} seconds'.format(time.time()-start_time))
    train_time = time.time()-start_time
    
    start_time = time.time()
    test_preds = lasso_model_w.predict(feature_test)
    print('Prediction time: {:4f} seconds'.format(time.time()-start_time))
    prediction_time = time.time()-start_time
    
    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    accuracy = 1-classification_error
    balanced_accuracy = balanced_accuracy_score(label_test,test_preds)
    test_probs = lasso_model_w.predict_proba(feature_test)[:,1]
    auc = roc_auc_score(label_test,test_probs)
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy_score(label_test,test_preds)))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))
    
    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_probs)))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train,
        'Feature Extraction Test Time':tm_feature_test,
        'Train Time':train_time,
        'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='Weighted Lasso')
    model_results_df = model_results_df.append(row)

Training time: 59.915938 seconds
Prediction time: 0.017789 seconds

Classification Error: 0.168333
Accuracy: 0.831667

Balanced Accuracy: 0.622522
Classification Report:

              precision    recall  f1-score   support

           0       0.83      0.99      0.90       473
           1       0.82      0.26      0.40       127

    accuracy                           0.83       600
   macro avg       0.83      0.62      0.65       600
weighted avg       0.83      0.83      0.79       600

Confusion Matrix:

[[466   7]
 [ 94  33]]

AUC: 0.819414


### SMOTE Bagging

In [40]:
#params = {'n_estimators':[25,50,75,100]}
#cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
#gscv = GridSearchCV(BaggingClassifier(),params,cv=cv,scoring='roc_auc').fit(feature_train_sm,label_train_sm)
#gscv.best_params_
#output: {{'n_estimators': 100}}

In [41]:
if run_bagging_smote == True:
    
    tm_feature_train = np.nan
    start = time.time()
    dat_train = feature_improved(fiducial_pt_list, train_idx, info)
    feature_train = dat_train.loc[:, dat_train.columns != 'labels']
    label_train = dat_train['labels']
    over = SMOTE(sampling_strategy='auto')
    under = RandomUnderSampler(sampling_strategy='auto')
    sm = Pipeline(steps = [('o', over), ('u', under)])
    feature_train_sm, label_train_sm = sm.fit_resample(feature_train,label_train)
    end = time.time()
    tm_feature_train = end-start
         
    tm_feature_test = np.nan
    start = time.time()
    dat_test = feature_improved(fiducial_pt_list, test_idx, info)
    feature_test = dat_test.loc[:, dat_test.columns != 'labels']
    label_test = dat_test['labels']
    end = time.time()
    tm_feature_test = end-start
    
    
    start_time=time.time()
    bagging_smote = BaggingClassifier(n_estimators = 100)
    bagging_smote.fit(feature_train_sm, label_train_sm)
    train_time = time.time()-start_time
    print('Training time: {:4f} seconds'.format(time.time()-start_time))

    start_time = time.time()
    test_preds = bagging_smote.predict(feature_test)
    prediction_time = time.time()-start_time
    print('Prediction time: {:4f} seconds'.format(time.time()-start_time))

    classification_error = np.mean(np.array(test_preds) != np.array(label_test))
    accuracy = 1-classification_error
    balanced_accuracy = balanced_accuracy_score(label_test,test_preds)
    test_probs = bagging_smote.predict_proba(feature_test)[:,1]
    auc = roc_auc_score(label_test,test_probs)
    
    print('\nClassification Error: {:4f}'.format(classification_error)) 
    print('Accuracy: {:4f}\n'.format(1-classification_error))
    print('Balanced Accuracy: {:4f}'.format(balanced_accuracy_score(label_test,test_preds)))
    print('Classification Report:\n')
    print(classification_report(label_test,test_preds))
    print('Confusion Matrix:\n')
    print(confusion_matrix(label_test,test_preds))

    #Note: AUC is a better metric than accuracy because of imbalanced classes
    print('\nAUC: {:4f}'.format(roc_auc_score(label_test,test_probs)))
    
    row = pd.Series({'Feature Extraction Train Time':tm_feature_train,
        'Feature Extraction Test Time':tm_feature_test,
        'Train Time':train_time,
        'Prediction Time':prediction_time,
        'Accuracy':accuracy,
        'AUC':auc,
        'Balanced Accuracy':balanced_accuracy},name='SMOTE Bagging')
    model_results_df = model_results_df.append(row)

Training time: 648.656514 seconds
Prediction time: 0.470589 seconds

Classification Error: 0.198333
Accuracy: 0.801667

Balanced Accuracy: 0.629414
Classification Report:

              precision    recall  f1-score   support

           0       0.84      0.93      0.88       473
           1       0.55      0.33      0.41       127

    accuracy                           0.80       600
   macro avg       0.70      0.63      0.65       600
weighted avg       0.78      0.80      0.78       600

Confusion Matrix:

[[439  34]
 [ 85  42]]

AUC: 0.777122


After undersampling/oversampling, we now have equal number of members in each class

In [42]:
print('Number of records with label 0 (basic emotion):   {:4d} '.format(len(label_train_sm)-sum(label_train_sm)))
print('Number of records with label 1 (complex emotion): {:2d} '.format(sum(label_train_sm)))

Number of records with label 0 (basic emotion):   1929 
Number of records with label 1 (complex emotion): 1929 


### 10-fold Cross Validation 

In [43]:
# weighted lasso
#print("Cross Validation Score: ", np.mean(cross_val_score(lasso_w, feature_train, label_train, cv=10, scoring='roc_auc')))
# output:0.8445833333333332
# roc_auc output:0.8010359440647241

In [44]:
# weighted svm
#print("Cross Validation Score: ", np.mean(cross_val_score(svm, feature_train, label_train, cv=10, scoring='roc_auc')))
# output:0.8195833333333333
# roc_auc output:0.8117648633580459

In [45]:
# weighted Logistic
#print("Cross Validation Score: ", np.mean(cross_val_score(lr, feature_train, label_train, cv=10, scoring='roc_auc')))
# output:0.8300000000000001

In [46]:
# XGBoost with SMOTE
#print("Cross Validation Score: ", np.mean(cross_val_score(xgb_sm, feature_train, label_train, cv=10, scoring='roc_auc')))
# output:0.8300000000000001
# roc_auc output:0.8387998261113715

### Table

In [47]:
model_results_df

,Feature Extraction Train Time,Feature Extraction Test Time,Train Time,Prediction Time,Accuracy,Balanced Accuracy,AUC
Baseline,5.087896,1.252400,186.447609,0.050044,0.811667,0.595437,0.785237
Advanced (SMOTEBoost),2.307424,0.035263,207.430647,0.070880,0.823333,0.712274,0.825024
Baseline with Improved Features,0.148918,0.033018,178.205290,0.029718,0.815000,0.606191,0.798672
Baseline with PCA,8.117146,0.039645,1.319941,0.001212,0.786667,0.527742,0.691332
KNN,0.151489,0.035817,0.419009,6.096245,0.791667,0.516514,0.674527
SMOTE KNN,2.284506,0.029388,0.747403,8.509360,0.578333,0.623121,0.682376
XGBoost,0.152626,0.040548,0.040548,0.040548,0.813333,0.688652,0.808726
Random Forest,0.153098,0.032813,0.040548,0.040548,0.810000,0.574221,0.752093
LDA,0.155403,0.032234,0.040548,0.040548,0.821667,0.636339,0.786203
Logistic Regression,0.166774,0.031862,100.128016,0.024673,0.821667,0.699697,0.822310


In [48]:
model_results_df = model_results_df.applymap(lambda x: round(x,3))
model_results_df['Train Time'] = [str(x)+' s' for x in list(model_results_df['Train Time'])]
model_results_df['Prediction Time'] = [str(x)+' s' for x in list(model_results_df['Prediction Time'])]
model_results_df['Feature Extraction Train Time'] = [str(x)+' s' for x in list(model_results_df['Feature Extraction Train Time'])]
model_results_df['Feature Extraction Test Time'] = [str(x)+' s' for x in list(model_results_df['Feature Extraction Test Time'])]

In [49]:
model_results_df

,Feature Extraction Train Time,Feature Extraction Test Time,Train Time,Prediction Time,Accuracy,Balanced Accuracy,AUC
Baseline,5.088 s,1.252 s,186.448 s,0.05 s,0.812,0.595,0.785
Advanced (SMOTEBoost),2.307 s,0.035 s,207.431 s,0.071 s,0.823,0.712,0.825
Baseline with Improved Features,0.149 s,0.033 s,178.205 s,0.03 s,0.815,0.606,0.799
Baseline with PCA,8.117 s,0.04 s,1.32 s,0.001 s,0.787,0.528,0.691
KNN,0.151 s,0.036 s,0.419 s,6.096 s,0.792,0.517,0.675
SMOTE KNN,2.285 s,0.029 s,0.747 s,8.509 s,0.578,0.623,0.682
XGBoost,0.153 s,0.041 s,0.041 s,0.041 s,0.813,0.689,0.809
Random Forest,0.153 s,0.033 s,0.041 s,0.041 s,0.810,0.574,0.752
LDA,0.155 s,0.032 s,0.041 s,0.041 s,0.822,0.636,0.786
Logistic Regression,0.167 s,0.032 s,100.128 s,0.025 s,0.822,0.700,0.822
